In [1]:
%matplotlib inline

In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import rcParams

import numpy as np
import pandas as pd


import shapely.geometry as sgeom

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

import geopandas as gpd


paper_path = Path('../../paper/figures/')

slide_path = Path('../../slides/figures/intro/')
proposal_path = Path('../../draft/figures/intro/')
#rcParams['font.family'] = 'Segoe Print'

In [3]:
YEAR = 2021
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

In [4]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [5]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [9]:
nydf = df[df['STATE'].str.match("NY")]

In [10]:
ds['ELEMENT'].unique().compute()

C:\Users\story\Anaconda3\envs\mplrelease\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


0     TMAX
1     PRCP
2     TAVG
3     TMIN
4     SNWD
      ... 
59    PSUN
60    TSUN
61    DWPR
62    WT07
63    WT10
Name: ELEMENT, Length: 64, dtype: object

In [11]:
nyds = ds[ds['ID'].isin(nydf['ID'].unique()) & ds['ELEMENT'].isin(['TAVG', 'PRCP' ])].compute()

C:\Users\story\Anaconda3\envs\mplrelease\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


In [12]:
ny = nyds.merge(nydf, on='ID')

In [13]:
ny.columns

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
       'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [15]:
ds['ELEMENT'].unique().compute()

C:\Users\story\Anaconda3\envs\mplrelease\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


0     TMAX
1     PRCP
2     TAVG
3     TMIN
4     SNWD
      ... 
59    PSUN
60    TSUN
61    DWPR
62    WT07
63    WT10
Name: ELEMENT, Length: 64, dtype: object

In [16]:
nyd = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [17]:
nyd

NAME LATITUDE LONGITUDE       DATE DATA_VALUE     
ELEMENT                                                        PRCP TAVG
0           ALBANY 0.3 ESE  42.6641  -73.7935 2021-03-31          0  NaN
1           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-01         64  NaN
2           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-02         97  NaN
3           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-03          0  NaN
4           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-04          0  NaN
...                    ...      ...       ...        ...        ...  ...
47701    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-09         28  NaN
47702    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-10          0  NaN
47703    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-11         86  NaN
47704    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-12         41  NaN
47705    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-13         18  NaN

[47706 rows x 6 columns]

In [18]:
nyd.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyd.columns.to_flat_index())]

In [19]:
nyd.head()

,NAME,LATITUDE,LONGITUDE,DATE,PRCP,TAVG
0,ALBANY 0.3 ESE,42.6641,-73.7935,2021-03-31,0,NaN
1,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-01,64,NaN
2,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-02,97,NaN
3,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-03,0,NaN
4,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-04,0,NaN


In [20]:
nyd['TAVGF'] =  (nyd['TAVG'].astype(float)/10) *(9/5) + 32

In [21]:
nyd.head()

,NAME,LATITUDE,LONGITUDE,DATE,PRCP,TAVG,TAVGF
0,ALBANY 0.3 ESE,42.6641,-73.7935,2021-03-31,0,NaN,NaN
1,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-01,64,NaN,NaN
2,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-02,97,NaN,NaN
3,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-03,0,NaN,NaN
4,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-04,0,NaN,NaN


In [28]:
nyd['PRCP'].astype(float).describe()

count    47142.000000
mean        20.651775
std         50.864002
min          0.000000
25%          0.000000
50%          0.000000
75%         13.000000
max        759.000000
Name: PRCP, dtype: float64

In [32]:
nyd['PRCPI'] = nyd['PRCP'].astype(float)/10 * 0.039370 

In [33]:
nyd['PRCPI'].describe()

count    47142.000000
mean         0.081306
std          0.200252
min          0.000000
25%          0.000000
50%          0.000000
75%          0.051181
max          2.988183
Name: PRCPI, dtype: float64

In [34]:
nyd.to_csv("nydata", index=False)